In [1]:
import sys
import os

try:
    os.chdir(r"/home/hanaz63/PAPER_MOLECULAR_ROTATIONS_2022/nobackup/molgri")
    sys.path.append(r"/home/hanaz63/PAPER_MOLECULAR_ROTATIONS_2022/nobackup")
except FileNotFoundError:
    os.chdir(r"D:\HANA\phD\PAPER_2022\molecularRotationalGrids")
    sys.path.append(r"D:\HANA\phD\PAPER_2022\molecularRotationalGrids")
    
import warnings
warnings.filterwarnings("ignore")

Erstmals: wir brauchen eine HF Monomer Datai.

In [5]:
%pycat input/HF.xyz

2
a simple HF molecule in gas phase, units A
H  0  0  0
F  0.92  0  0


Nun lass zwei PTs (klein, groß) produzieren:
- klein: -o 42 -t "linspace(0.1, 0.4, 10)" -b 8
- groß: -o 162 -t "linspace(0.1, 0.4, 20)" -b 40

In [17]:
%run molgri/scripts/generate_pt.py -m1 HF -m2 HF -o 42 -t "linspace(0.1, 0.4, 10)" -b 8

Saved the log file to output/data/logging/HF_HF_0000.log
Saved pseudo-trajectory to output/data/pt_files/HF_HF_0000.xtc and structure file to output/data/pt_files/HF_HF_0000.gro
Timing the generation of Pt HF_HF_b_cube4D_8_o_ico_42_t_965211660: 0:00:05.554178 hours:minutes:seconds


In [18]:
%run molgri/scripts/generate_pt.py -m1 HF -m2 HF -o 162 -t "linspace(0.1, 0.4, 20)" -b 40

Saved the log file to output/data/logging/HF_HF_0001.log
Saved pseudo-trajectory to output/data/pt_files/HF_HF_0001.xtc and structure file to output/data/pt_files/HF_HF_0001.gro
Timing the generation of Pt HF_HF_b_cube4D_40_o_ico_162_t_4038933404: 0:03:41.259992 hours:minutes:seconds


In [2]:
import nglview as nv
import MDAnalysis as mda

In [5]:
u = mda.Universe("output/data/pt_files/HF_HF_0000.gro", "output/data/pt_files/HF_HF_0000.xtc")

protein = u.select_atoms('all')
w = nv.show_mdanalysis(protein)
w

NGLWidget(max_frame=3359)

In [3]:
nv.demo()

NGLWidget()

In [4]:
import ipywidgets 
ipywidgets.Text("hello")

Text(value='hello')